## Import Libraries

In [1]:
import pandas as pd # for data analsysis
import numpy as np # to handle data in a vectorized manner

import requests
!pip install geocoder
import geocoder
from geopy.geocoders import Nominatim # module to convert an address into latitude and longitude values

# libraries for displaying images
from IPython.display import Image 
from IPython.core.display import HTML 
    
#tranforming json file into a pandas dataframe library
from pandas.io.json import json_normalize

!pip install folium
import folium # plotting library

print('Libraries Imported')

Libraries Imported


## Define Foursquare Credentials

In [2]:
CLIENT_ID = '4FQQJE2LMGVLERRRE2BOFR1LQQR5ZEHFLTD1WRAFKOFT4TI0' # your Foursquare ID
CLIENT_SECRET = 'RCSNR5TH31NQRG2QYCHK101HRMTUZHBB2TSJ1CJU5UOGRXRQ' # your Foursquare Secret
VERSION = '20180604'
LIMIT =30
print('Your credentails:')
print('Foursquare_ID: ' + CLIENT_ID)
print('Foursquare_Secret:' + CLIENT_SECRET)

Your credentails:
Foursquare_ID: 4FQQJE2LMGVLERRRE2BOFR1LQQR5ZEHFLTD1WRAFKOFT4TI0
Foursquare_Secret:RCSNR5TH31NQRG2QYCHK101HRMTUZHBB2TSJ1CJU5UOGRXRQ


## Define the City and get its Latitude & Longitude

In [3]:
# define the city and get its latitude & longitude 
city = 'Toronto'
geolocator = Nominatim(user_agent="foursquare_agent")
location = geolocator.geocode(city)
latitude = location.latitude
longitude = location.longitude
print(latitude, longitude)

43.6534817 -79.3839347


## Search for Hotels

In [4]:
# search for hotels
search_query = 'Hotel'
radius = 500

# Define the corresponding URL
url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&query={}&radius={}&limit={}'\
.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude, VERSION, search_query, radius, LIMIT)
url

'https://api.foursquare.com/v2/venues/search?client_id=4FQQJE2LMGVLERRRE2BOFR1LQQR5ZEHFLTD1WRAFKOFT4TI0&client_secret=RCSNR5TH31NQRG2QYCHK101HRMTUZHBB2TSJ1CJU5UOGRXRQ&ll=43.6534817,-79.3839347&v=20180604&query=Hotel&radius=500&limit=30'

In [5]:
# Send the GET Request and examine the results
results = requests.get(url).json()
#results

In [6]:
# assign relevant part of JSON to venues
venues = results['response']['venues']

# tranform venues into a dataframe
dataframe = json_normalize(venues)
dataframe.head()

,categories,hasPerk,id,location.address,location.cc,location.city,location.country,location.crossStreet,location.distance,location.formattedAddress,location.labeledLatLngs,location.lat,location.lng,location.neighborhood,location.postalCode,location.state,name,referralId,venuePage.id
0,"[{'id': '4bf58dd8d48988d1fa931735', 'name': 'H...",False,4ab2d511f964a5209b6c20e3,123 Queen Street West,CA,Toronto,Canada,at York St.,324,"[123 Queen Street West (at York St.), Toronto ...","[{'label': 'display', 'lat': 43.6505944, 'lng'...",43.650594,-79.384530,NaN,M5H 2M9,ON,Sheraton Centre Toronto Hotel,v-1589118152,NaN
1,"[{'id': '4bf58dd8d48988d1fa931735', 'name': 'H...",False,51d212c3498ebf27dc469bc9,33 Gerrard Street West,CA,Toronto,Canada,at Yonge St,562,"[33 Gerrard Street West (at Yonge St), Toronto...","[{'label': 'display', 'lat': 43.65849759157591...",43.658498,-79.383097,NaN,M5G 1Z4,ON,Chelsea Hotel,v-1589118152,NaN
2,"[{'id': '4bf58dd8d48988d1fa931735', 'name': 'H...",False,4ae61cf6f964a520caa421e3,200 Victoria St,CA,Toronto,Canada,at Shuter St,410,"[200 Victoria St (at Shuter St), Toronto ON, C...","[{'label': 'display', 'lat': 43.65449797039222...",43.654498,-79.379035,NaN,NaN,ON,Pantages Hotel & Spa,v-1589118152,NaN
3,"[{'id': '4bf58dd8d48988d1fa931735', 'name': 'H...",False,51e48697498eded9073c6c17,33 Gerrard Street West,CA,Toronto,Canada,NaN,522,"[33 Gerrard Street West, Toronto ON, Canada]","[{'label': 'display', 'lat': 43.65809435580944...",43.658094,-79.382711,NaN,NaN,ON,Crew Room Eaton Chelsea Hotel,v-1589118152,NaN
4,"[{'id': '4bf58dd8d48988d1e7931735', 'name': 'J...",False,4b68aed1f964a520de862be3,194 Queen St W,CA,Toronto,Canada,Queen & St. Patrick,499,"[194 Queen St W (Queen & St. Patrick), Toronto...","[{'label': 'display', 'lat': 43.65050475544005...",43.650505,-79.388577,NaN,M5V 1Z1,ON,The Rex Hotel Jazz & Blues Bar,v-1589118152,62225795


## Clean Hotel Dataframe

In [7]:
# keep only columns that include venue name, and anything that is associated with location
clean_columns = ['name', 'categories'] + [col for col in dataframe.columns if col.startswith('location.')]+ ['id']
clean_dataframe = dataframe.loc[:,clean_columns]

# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

# filter the category for each row
clean_dataframe['categories'] = clean_dataframe.apply(get_category_type, axis=1)

# clean column names by keeping only last term
clean_dataframe.columns = [column.split('.')[-1] for column in clean_dataframe.columns]

clean_dataframe.head()

,name,categories,address,cc,city,country,crossStreet,distance,formattedAddress,labeledLatLngs,lat,lng,neighborhood,postalCode,state,id
0,Sheraton Centre Toronto Hotel,Hotel,123 Queen Street West,CA,Toronto,Canada,at York St.,324,"[123 Queen Street West (at York St.), Toronto ...","[{'label': 'display', 'lat': 43.6505944, 'lng'...",43.650594,-79.384530,NaN,M5H 2M9,ON,4ab2d511f964a5209b6c20e3
1,Chelsea Hotel,Hotel,33 Gerrard Street West,CA,Toronto,Canada,at Yonge St,562,"[33 Gerrard Street West (at Yonge St), Toronto...","[{'label': 'display', 'lat': 43.65849759157591...",43.658498,-79.383097,NaN,M5G 1Z4,ON,51d212c3498ebf27dc469bc9
2,Pantages Hotel & Spa,Hotel,200 Victoria St,CA,Toronto,Canada,at Shuter St,410,"[200 Victoria St (at Shuter St), Toronto ON, C...","[{'label': 'display', 'lat': 43.65449797039222...",43.654498,-79.379035,NaN,NaN,ON,4ae61cf6f964a520caa421e3
3,Crew Room Eaton Chelsea Hotel,Hotel,33 Gerrard Street West,CA,Toronto,Canada,NaN,522,"[33 Gerrard Street West, Toronto ON, Canada]","[{'label': 'display', 'lat': 43.65809435580944...",43.658094,-79.382711,NaN,NaN,ON,51e48697498eded9073c6c17
4,The Rex Hotel Jazz & Blues Bar,Jazz Club,194 Queen St W,CA,Toronto,Canada,Queen & St. Patrick,499,"[194 Queen St W (Queen & St. Patrick), Toronto...","[{'label': 'display', 'lat': 43.65050475544005...",43.650505,-79.388577,NaN,M5V 1Z1,ON,4b68aed1f964a520de862be3


In [8]:
# delete unnecessary columns
clean_dataframe2= clean_dataframe.drop(['cc', 'city', 'country', 'crossStreet', 'distance', 'formattedAddress',\
                                        'labeledLatLngs','neighborhood', 'id'], axis=1)
clean_dataframe2

,name,categories,address,lat,lng,postalCode,state
0,Sheraton Centre Toronto Hotel,Hotel,123 Queen Street West,43.650594,-79.384530,M5H 2M9,ON
1,Chelsea Hotel,Hotel,33 Gerrard Street West,43.658498,-79.383097,M5G 1Z4,ON
2,Pantages Hotel & Spa,Hotel,200 Victoria St,43.654498,-79.379035,NaN,ON
3,Crew Room Eaton Chelsea Hotel,Hotel,33 Gerrard Street West,43.658094,-79.382711,NaN,ON
4,The Rex Hotel Jazz & Blues Bar,Jazz Club,194 Queen St W,43.650505,-79.388577,M5V 1Z1,ON
5,VFM Test Hotel,Hotel,123 Test Drive,43.658434,-79.387894,M2M 2M2,ON
6,One King West Hotel & Residence,Hotel,1 King St W,43.649139,-79.377876,M5H 1A1,ON
7,Cosmopolitan Toronto Centre Hotel & Spa,Hotel,8 Colborne St,43.649064,-79.377598,M5E 1E1,ON
8,Sheraton Centre Hotel Club Lounge,Lounge,NaN,43.651063,-79.384527,M5H 2M9,ON
9,Stathcona Hotel,Hotel,NaN,43.654947,-79.386359,NaN,ON


In [9]:
# delete rows with none values
clean_dataframe3 = clean_dataframe2.dropna(axis=0, how='any', thresh=None, subset=None, inplace=False)
clean_dataframe3

,name,categories,address,lat,lng,postalCode,state
0,Sheraton Centre Toronto Hotel,Hotel,123 Queen Street West,43.650594,-79.384530,M5H 2M9,ON
1,Chelsea Hotel,Hotel,33 Gerrard Street West,43.658498,-79.383097,M5G 1Z4,ON
4,The Rex Hotel Jazz & Blues Bar,Jazz Club,194 Queen St W,43.650505,-79.388577,M5V 1Z1,ON
5,VFM Test Hotel,Hotel,123 Test Drive,43.658434,-79.387894,M2M 2M2,ON
6,One King West Hotel & Residence,Hotel,1 King St W,43.649139,-79.377876,M5H 1A1,ON
7,Cosmopolitan Toronto Centre Hotel & Spa,Hotel,8 Colborne St,43.649064,-79.377598,M5E 1E1,ON
12,Pantages Hotel Gym,Gym,200 Victoria St.,43.654464,-79.379036,M5B 1V8,ON
16,Pantages Hotel Toronto Centre,Residential Building (Apartment / Condo),200 Victoria Street,43.655205,-79.379182,M5B 1V8,ON
17,The Spa At The Adelaide Hotel,Spa,325 Bay St,43.649725,-79.380346,M5H 4G5,ON
20,"Chelsea Hotel, Toronto Meetings & Events Venue",Convention Center,33 Gerrard Street West,43.658446,-79.382985,M5G 1Z4,ON


In [10]:
# delete rows which its category is not Hotel or Event Space
array= ['Hotel', 'Event Space']
hotel_dataframe= clean_dataframe3.loc[clean_dataframe3['categories'].isin(array)]
hotel_dataframe

,name,categories,address,lat,lng,postalCode,state
0,Sheraton Centre Toronto Hotel,Hotel,123 Queen Street West,43.650594,-79.384530,M5H 2M9,ON
1,Chelsea Hotel,Hotel,33 Gerrard Street West,43.658498,-79.383097,M5G 1Z4,ON
5,VFM Test Hotel,Hotel,123 Test Drive,43.658434,-79.387894,M2M 2M2,ON
6,One King West Hotel & Residence,Hotel,1 King St W,43.649139,-79.377876,M5H 1A1,ON
7,Cosmopolitan Toronto Centre Hotel & Spa,Hotel,8 Colborne St,43.649064,-79.377598,M5E 1E1,ON
22,St Regis Hotel,Hotel,325 Bay St,43.649770,-79.380701,M5H 4G3,ON
23,The Adelaide Hotel Toronto,Hotel,325 Bay St,43.649831,-79.380164,M5H 4G3,ON
25,DoubleTree by Hilton,Hotel,108 Chestnut Street,43.654608,-79.385942,M5G 1R3,ON
28,Marriott Downtown at CF Toronto Eaton Centre,Hotel,525 Bay Street,43.654728,-79.382422,M5G 2L2,ON


In [11]:
# delete rows which has duplicate hotel's name
df_hotels = hotel_dataframe.drop_duplicates(subset='name', keep="first")
df_hotels

,name,categories,address,lat,lng,postalCode,state
0,Sheraton Centre Toronto Hotel,Hotel,123 Queen Street West,43.650594,-79.384530,M5H 2M9,ON
1,Chelsea Hotel,Hotel,33 Gerrard Street West,43.658498,-79.383097,M5G 1Z4,ON
5,VFM Test Hotel,Hotel,123 Test Drive,43.658434,-79.387894,M2M 2M2,ON
6,One King West Hotel & Residence,Hotel,1 King St W,43.649139,-79.377876,M5H 1A1,ON
7,Cosmopolitan Toronto Centre Hotel & Spa,Hotel,8 Colborne St,43.649064,-79.377598,M5E 1E1,ON
22,St Regis Hotel,Hotel,325 Bay St,43.649770,-79.380701,M5H 4G3,ON
23,The Adelaide Hotel Toronto,Hotel,325 Bay St,43.649831,-79.380164,M5H 4G3,ON
25,DoubleTree by Hilton,Hotel,108 Chestnut Street,43.654608,-79.385942,M5G 1R3,ON
28,Marriott Downtown at CF Toronto Eaton Centre,Hotel,525 Bay Street,43.654728,-79.382422,M5G 2L2,ON


In [12]:
# choose the hotel which has the same postalCode with the event space
df_hotel = df_hotels[df_hotels.postalCode == 'M5H 2M9']
df_hotel

,name,categories,address,lat,lng,postalCode,state
0,Sheraton Centre Toronto Hotel,Hotel,123 Queen Street West,43.650594,-79.38453,M5H 2M9,ON


## Search for Parks

In [13]:
# search for Parks
search_query = 'Park'
radius = 10000

# Define the corresponding URL
url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&query={}&radius={}&limit={}'\
.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude, VERSION, search_query, radius, LIMIT)
url

'https://api.foursquare.com/v2/venues/search?client_id=4FQQJE2LMGVLERRRE2BOFR1LQQR5ZEHFLTD1WRAFKOFT4TI0&client_secret=RCSNR5TH31NQRG2QYCHK101HRMTUZHBB2TSJ1CJU5UOGRXRQ&ll=43.6534817,-79.3839347&v=20180604&query=Park&radius=10000&limit=30'

In [14]:
# Send the GET Request and examine the results
presults = requests.get(url).json()
#presults

In [15]:
# assign relevant part of JSON to venues
venues = presults['response']['venues']

# tranform venues into a dataframe
park_dataframe = json_normalize(venues)
park_dataframe.head()

,categories,hasPerk,id,location.address,location.cc,location.city,location.country,location.crossStreet,location.distance,location.formattedAddress,location.labeledLatLngs,location.lat,location.lng,location.postalCode,location.state,name,referralId,venuePage.id
0,"[{'id': '4bf58dd8d48988d1fd931735', 'name': 'M...",False,4b170e47f964a5208cc123e3,691 University Ave,CA,Toronto,Canada,at College St,887,"[691 University Ave (at College St), Toronto O...","[{'label': 'display', 'lat': 43.66000635700919...",43.660006,-79.390272,M5G 2P1,ON,Queen's Park Subway Station,v-1589118152,NaN
1,"[{'id': '4bf58dd8d48988d163941735', 'name': 'P...",False,4b54deadf964a520a3d027e3,NaN,CA,Toronto,Canada,NaN,663,"[btwn College, University, Queen & Spadina, To...","[{'label': 'display', 'lat': 43.65248755299856...",43.652488,-79.392053,NaN,ON,Grange Park,v-1589118152,NaN
2,"[{'id': '4bf58dd8d48988d163941735', 'name': 'P...",False,4bfc5019da7120a132df49fd,450 Adelaide Street West,CA,Toronto,Canada,Brant St & Adelaide St W,1372,[450 Adelaide Street West (Brant St & Adelaide...,"[{'label': 'display', 'lat': 43.64738759338116...",43.647388,-79.398752,NaN,ON,St. Andrews Playground / Dog Park,v-1589118152,NaN
3,"[{'id': '4bf58dd8d48988d1fc931735', 'name': 'L...",False,4b9177e4f964a520d2be33e3,NaN,CA,Toronto,Canada,NaN,389,"[Toronto ON, Canada]","[{'label': 'display', 'lat': 43.650735, 'lng':...",43.650735,-79.386921,NaN,ON,TTC Streetcar #403 - Victoria park,v-1589118152,NaN
4,"[{'id': '4d954b06a243a5684965b473', 'name': 'R...",False,4ad80fcef964a520a61021e3,280 Simcoe St,CA,Toronto,Canada,Dundas Street West,539,"[280 Simcoe St (Dundas Street West), Toronto O...","[{'label': 'display', 'lat': 43.65544241747622...",43.655442,-79.390056,M5T 2Y5,ON,One Park Lane,v-1589118152,NaN


## Clean Park Dataframe

In [16]:
# keep only columns that include venue name, and anything that is associated with location
park_clean_columns = ['name', 'categories'] + [col for col in park_dataframe.columns if col.startswith('location.')]+ ['id']
clean_park_dataframe = park_dataframe.loc[:,park_clean_columns]

# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list1 = row['categories']
    except:
        categories_list1 = row['venue.categories']
        
    if len(categories_list1) == 0:
        return None
    else:
        return categories_list1[0]['name']

# filter the category for each row
clean_park_dataframe['categories'] = clean_park_dataframe.apply(get_category_type, axis=1)

# clean column names by keeping only last term
clean_park_dataframe.columns = [column.split('.')[-1] for column in clean_park_dataframe.columns]

clean_park_dataframe.head()

,name,categories,address,cc,city,country,crossStreet,distance,formattedAddress,labeledLatLngs,lat,lng,postalCode,state,id
0,Queen's Park Subway Station,Metro Station,691 University Ave,CA,Toronto,Canada,at College St,887,"[691 University Ave (at College St), Toronto O...","[{'label': 'display', 'lat': 43.66000635700919...",43.660006,-79.390272,M5G 2P1,ON,4b170e47f964a5208cc123e3
1,Grange Park,Park,NaN,CA,Toronto,Canada,NaN,663,"[btwn College, University, Queen & Spadina, To...","[{'label': 'display', 'lat': 43.65248755299856...",43.652488,-79.392053,NaN,ON,4b54deadf964a520a3d027e3
2,St. Andrews Playground / Dog Park,Park,450 Adelaide Street West,CA,Toronto,Canada,Brant St & Adelaide St W,1372,[450 Adelaide Street West (Brant St & Adelaide...,"[{'label': 'display', 'lat': 43.64738759338116...",43.647388,-79.398752,NaN,ON,4bfc5019da7120a132df49fd
3,TTC Streetcar #403 - Victoria park,Light Rail Station,NaN,CA,Toronto,Canada,NaN,389,"[Toronto ON, Canada]","[{'label': 'display', 'lat': 43.650735, 'lng':...",43.650735,-79.386921,NaN,ON,4b9177e4f964a520d2be33e3
4,One Park Lane,Residential Building (Apartment / Condo),280 Simcoe St,CA,Toronto,Canada,Dundas Street West,539,"[280 Simcoe St (Dundas Street West), Toronto O...","[{'label': 'display', 'lat': 43.65544241747622...",43.655442,-79.390056,M5T 2Y5,ON,4ad80fcef964a520a61021e3


In [17]:
# delete unnecessary columns
clean_park_dataframe2= clean_park_dataframe.drop(['cc', 'city', 'country', 'crossStreet', 'distance', 'formattedAddress',\
                                        'labeledLatLngs', 'id'], axis=1)
clean_park_dataframe2

,name,categories,address,lat,lng,postalCode,state
0,Queen's Park Subway Station,Metro Station,691 University Ave,43.660006,-79.390272,M5G 2P1,ON
1,Grange Park,Park,NaN,43.652488,-79.392053,NaN,ON
2,St. Andrews Playground / Dog Park,Park,450 Adelaide Street West,43.647388,-79.398752,NaN,ON
3,TTC Streetcar #403 - Victoria park,Light Rail Station,NaN,43.650735,-79.386921,NaN,ON
4,One Park Lane,Residential Building (Apartment / Condo),280 Simcoe St,43.655442,-79.390056,M5T 2Y5,ON
5,Physiomed College Park,Medical Center,382 Yonge St,43.659240,-79.382860,M5B 1S8,ON
6,Queen's Park,Park,University Ave.,43.663946,-79.392180,M5R 2E8,ON
7,Berczy Park,Park,35 Wellington St. East,43.648048,-79.375172,NaN,ON
8,Residences at College Park North,Residential Building (Apartment / Condo),763 Bay St.,43.659822,-79.385159,M5G 2R3,ON
9,Toothworks - College Park Dental,Dentist's Office,"444 Yonge St, Unit M16",43.660009,-79.383905,NaN,ON


In [18]:
# delete rows with none values
clean_park_dataframe3 = clean_park_dataframe2.dropna(axis=0, how='any', thresh=None, subset=None, inplace=False)
clean_park_dataframe3

,name,categories,address,lat,lng,postalCode,state
0,Queen's Park Subway Station,Metro Station,691 University Ave,43.660006,-79.390272,M5G 2P1,ON
4,One Park Lane,Residential Building (Apartment / Condo),280 Simcoe St,43.655442,-79.390056,M5T 2Y5,ON
5,Physiomed College Park,Medical Center,382 Yonge St,43.659240,-79.382860,M5B 1S8,ON
6,Queen's Park,Park,University Ave.,43.663946,-79.392180,M5R 2E8,ON
8,Residences at College Park North,Residential Building (Apartment / Condo),763 Bay St.,43.659822,-79.385159,M5G 2R3,ON
10,Crunch Fitness - College Park,Gym / Fitness Center,382 Yonge Street,43.659263,-79.382632,M5G 1S8,ON
12,Riverdale Park West,Park,500 Gerrard St.,43.666048,-79.360941,M5A 2H3,ON
15,Trinity Bellwoods Park,Park,1053 Dundas St. W.,43.647072,-79.413756,M5H 2N2,ON
19,Riverdale Park East,Park,550 Broadview Ave,43.669951,-79.355493,M4K 2P1,ON
22,Regent Park Community Centre,Community Center,465 Dundas St E,43.659512,-79.363048,M5A 2B2,ON


In [19]:
# delete rows which its category is not Park
df_park = clean_park_dataframe3[clean_park_dataframe3.categories == 'Park']
df_park

,name,categories,address,lat,lng,postalCode,state
6,Queen's Park,Park,University Ave.,43.663946,-79.392180,M5R 2E8,ON
12,Riverdale Park West,Park,500 Gerrard St.,43.666048,-79.360941,M5A 2H3,ON
15,Trinity Bellwoods Park,Park,1053 Dundas St. W.,43.647072,-79.413756,M5H 2N2,ON
19,Riverdale Park East,Park,550 Broadview Ave,43.669951,-79.355493,M4K 2P1,ON
23,Coronation Park,Park,711 Lake Shore Blvd. W,43.634553,-79.405403,M5V 3T7,ON
24,Barbara Hall Park,Park,519 Church St,43.666879,-79.381068,M4Y 2K9,ON
29,High Park,Park,1873 Bloor St. W,43.646479,-79.463425,M6R 2Z3,ON


## Search for Restaurants

In [20]:
# search for Restaurants
search_query = 'Restaurant'
radius = 10000

# Define the corresponding URL
url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&query={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude, VERSION, search_query, radius, LIMIT)
url

'https://api.foursquare.com/v2/venues/search?client_id=4FQQJE2LMGVLERRRE2BOFR1LQQR5ZEHFLTD1WRAFKOFT4TI0&client_secret=RCSNR5TH31NQRG2QYCHK101HRMTUZHBB2TSJ1CJU5UOGRXRQ&ll=43.6534817,-79.3839347&v=20180604&query=Restaurant&radius=10000&limit=30'

In [21]:
# Send the GET Request and examine the results
Rresults = requests.get(url).json()
#Rresults

In [22]:
# assign relevant part of JSON to venues
venues = Rresults['response']['venues']

# tranform venues into a dataframe
Restaurant_dataframe = json_normalize(venues)
Restaurant_dataframe.head()

,categories,hasPerk,id,location.address,location.cc,location.city,location.country,location.crossStreet,location.distance,location.formattedAddress,location.labeledLatLngs,location.lat,location.lng,location.neighborhood,location.postalCode,location.state,name,referralId,venuePage.id
0,"[{'id': '4bf58dd8d48988d14e941735', 'name': 'A...",False,4ad4c05ff964a52048f720e3,110 Chestnut Street,CA,Toronto,Canada,NaN,224,"[110 Chestnut Street, Toronto ON M5G 1R3, Canada]","[{'label': 'display', 'lat': 43.65488413420439...",43.654884,-79.385931,NaN,M5G 1R3,ON,Hemispheres Restaurant & Bistro,v-1589118153,NaN
1,"[{'id': '4bf58dd8d48988d1c4941735', 'name': 'R...",False,4ad4c05cf964a52006f620e3,37 King Street East,CA,Toronto,Canada,at Le Meridien King Edward Hotel,763,[37 King Street East (at Le Meridien King Edwa...,"[{'label': 'display', 'lat': 43.64929834396347...",43.649298,-79.376431,NaN,M5C 1E9,ON,Victoria's Restaurant,v-1589118153,498556908
2,"[{'id': '4bf58dd8d48988d14e941735', 'name': 'A...",False,4ada5d5bf964a520e92121e3,35 Church St,CA,Toronto,Canada,at Front St E,973,"[35 Church St (at Front St E), Toronto ON M5E ...","[{'label': 'display', 'lat': 43.64882370529773...",43.648824,-79.373702,NaN,M5E 1T3,ON,The Hot House Restaurant & Bar,v-1589118153,NaN
3,"[{'id': '4bf58dd8d48988d1f5931735', 'name': 'D...",False,4b072e9df964a52009f922e3,280 Spadina Ave.,CA,Toronto,Canada,at Dundas St. W.,1149,"[280 Spadina Ave. (at Dundas St. W.), Toronto ...","[{'label': 'display', 'lat': 43.65278331265585...",43.652783,-79.398174,NaN,NaN,ON,Sky Dragon Chinese Restaurant 龍翔酒樓,v-1589118153,NaN
4,"[{'id': '4bf58dd8d48988d143941735', 'name': 'B...",False,4bd47e6fcfa7b7139f2924da,389 Church St.,CA,Toronto,Canada,at Carlton St.,966,"[389 Church St. (at Carlton St.), Toronto ON M...","[{'label': 'display', 'lat': 43.66150015906530...",43.661500,-79.379319,NaN,M5B,ON,Studio Restaurant,v-1589118153,NaN


## Clean Restaurant Dataframe

In [23]:
# keep only columns that include venue name, and anything that is associated with location
Restaurant_clean_columns = ['name', 'categories'] + [col for col in Restaurant_dataframe.columns if col.startswith('location.')]+ ['id']
clean_Restaurant_dataframe = Restaurant_dataframe.loc[:,Restaurant_clean_columns]

# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list3 = row['categories']
    except:
        categories_list3 = row['venue.categories']
        
    if len(categories_list3) == 0:
        return None
    else:
        return categories_list3[0]['name']

# filter the category for each row
clean_Restaurant_dataframe['categories'] = clean_Restaurant_dataframe.apply(get_category_type, axis=1)

# clean column names by keeping only last term
clean_Restaurant_dataframe.columns = [column.split('.')[-1] for column in clean_Restaurant_dataframe.columns]

clean_Restaurant_dataframe.head()

,name,categories,address,cc,city,country,crossStreet,distance,formattedAddress,labeledLatLngs,lat,lng,neighborhood,postalCode,state,id
0,Hemispheres Restaurant & Bistro,American Restaurant,110 Chestnut Street,CA,Toronto,Canada,NaN,224,"[110 Chestnut Street, Toronto ON M5G 1R3, Canada]","[{'label': 'display', 'lat': 43.65488413420439...",43.654884,-79.385931,NaN,M5G 1R3,ON,4ad4c05ff964a52048f720e3
1,Victoria's Restaurant,Restaurant,37 King Street East,CA,Toronto,Canada,at Le Meridien King Edward Hotel,763,[37 King Street East (at Le Meridien King Edwa...,"[{'label': 'display', 'lat': 43.64929834396347...",43.649298,-79.376431,NaN,M5C 1E9,ON,4ad4c05cf964a52006f620e3
2,The Hot House Restaurant & Bar,American Restaurant,35 Church St,CA,Toronto,Canada,at Front St E,973,"[35 Church St (at Front St E), Toronto ON M5E ...","[{'label': 'display', 'lat': 43.64882370529773...",43.648824,-79.373702,NaN,M5E 1T3,ON,4ada5d5bf964a520e92121e3
3,Sky Dragon Chinese Restaurant 龍翔酒樓,Dim Sum Restaurant,280 Spadina Ave.,CA,Toronto,Canada,at Dundas St. W.,1149,"[280 Spadina Ave. (at Dundas St. W.), Toronto ...","[{'label': 'display', 'lat': 43.65278331265585...",43.652783,-79.398174,NaN,NaN,ON,4b072e9df964a52009f922e3
4,Studio Restaurant,Breakfast Spot,389 Church St.,CA,Toronto,Canada,at Carlton St.,966,"[389 Church St. (at Carlton St.), Toronto ON M...","[{'label': 'display', 'lat': 43.66150015906530...",43.661500,-79.379319,NaN,M5B,ON,4bd47e6fcfa7b7139f2924da


In [24]:
# delete unnecessary columns
clean_Restaurant_dataframe2= clean_Restaurant_dataframe.drop(['cc', 'city', 'country', 'crossStreet', 'distance', 'formattedAddress',\
                                        'labeledLatLngs', 'neighborhood', 'id'], axis=1)
clean_Restaurant_dataframe2.head()

,name,categories,address,lat,lng,postalCode,state
0,Hemispheres Restaurant & Bistro,American Restaurant,110 Chestnut Street,43.654884,-79.385931,M5G 1R3,ON
1,Victoria's Restaurant,Restaurant,37 King Street East,43.649298,-79.376431,M5C 1E9,ON
2,The Hot House Restaurant & Bar,American Restaurant,35 Church St,43.648824,-79.373702,M5E 1T3,ON
3,Sky Dragon Chinese Restaurant 龍翔酒樓,Dim Sum Restaurant,280 Spadina Ave.,43.652783,-79.398174,NaN,ON
4,Studio Restaurant,Breakfast Spot,389 Church St.,43.661500,-79.379319,M5B,ON


In [25]:
# delete rows with none values
df_Restaurant = clean_Restaurant_dataframe2.dropna(axis=0, how='any', thresh=None, subset=None, inplace=False)
df_Restaurant.head()

,name,categories,address,lat,lng,postalCode,state
0,Hemispheres Restaurant & Bistro,American Restaurant,110 Chestnut Street,43.654884,-79.385931,M5G 1R3,ON
1,Victoria's Restaurant,Restaurant,37 King Street East,43.649298,-79.376431,M5C 1E9,ON
2,The Hot House Restaurant & Bar,American Restaurant,35 Church St,43.648824,-79.373702,M5E 1T3,ON
4,Studio Restaurant,Breakfast Spot,389 Church St.,43.661500,-79.379319,M5B,ON
6,Azure Restaurant & Bar,Restaurant,225 Front St W,43.644749,-79.385113,M5V 2X3,ON


## Search for Cafeteria

In [26]:
# search for Cafeteria
search_query = 'Cafeteria'
radius = 10000

# Define the corresponding URL
url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&query={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude, VERSION, search_query, radius, LIMIT)
url

'https://api.foursquare.com/v2/venues/search?client_id=4FQQJE2LMGVLERRRE2BOFR1LQQR5ZEHFLTD1WRAFKOFT4TI0&client_secret=RCSNR5TH31NQRG2QYCHK101HRMTUZHBB2TSJ1CJU5UOGRXRQ&ll=43.6534817,-79.3839347&v=20180604&query=Cafeteria&radius=10000&limit=30'

In [27]:
# Send the GET Request and examine the results
cresults = requests.get(url).json()
#cresults

In [28]:
# assign relevant part of JSON to venues
venues = cresults['response']['venues']

# tranform venues into a dataframe
Cafeteria_dataframe = json_normalize(venues)
Cafeteria_dataframe.head()

,categories,hasPerk,id,location.address,location.cc,location.city,location.country,location.crossStreet,location.distance,location.formattedAddress,location.labeledLatLngs,location.lat,location.lng,location.postalCode,location.state,name,referralId
0,"[{'id': '4bf58dd8d48988d16e941735', 'name': 'F...",False,4f9165bde4b04fef0087e4e2,The Hospital for Sick Children (SickKids),CA,Toronto,Canada,NaN,448,"[The Hospital for Sick Children (SickKids), To...","[{'label': 'display', 'lat': 43.65720930549997...",43.657209,-79.386063,NaN,ON,Cafeteria,v-1589118153
1,"[{'id': '4bf58dd8d48988d142941735', 'name': 'A...",False,528e99a211d262edc3708a6f,388 Spadina Ave,CA,Toronto,Canada,at Nassau St,1246,"[388 Spadina Ave (at Nassau St), Toronto ON M5...","[{'label': 'display', 'lat': 43.65545427446646...",43.655454,-79.399163,M5T 2G5,ON,Stay Cafeteria 慢走,v-1589118153
2,"[{'id': '4bf58dd8d48988d120951735', 'name': 'F...",False,4b57256bf964a520332828e3,Yonge,CA,Toronto,Canada,Dundas,387,"[Yonge (Dundas), Toronto ON, Canada]","[{'label': 'display', 'lat': 43.65603785909391...",43.656038,-79.380672,NaN,ON,Sears Cafeteria,v-1589118153
3,"[{'id': '4bf58dd8d48988d128941735', 'name': 'C...",False,53bd6dfe498e6c1d3bc9333e,NaN,CA,Toronto,Canada,NaN,505,"[Toronto ON, Canada]","[{'label': 'display', 'lat': 43.655657, 'lng':...",43.655657,-79.389443,NaN,ON,Industrial Alliance Cafeteria,v-1589118153
4,"[{'id': '4bf58dd8d48988d128941735', 'name': 'C...",False,524f1c8f498e983dde61ada0,89 Chestnut Street,CA,Toronto,Canada,NaN,584,"[89 Chestnut Street, Toronto ON, Canada]","[{'label': 'display', 'lat': 43.648231, 'lng':...",43.648231,-79.384045,NaN,ON,Chestnut Tree Cafeteria,v-1589118153


## Clean Cafeteria Dataframe

In [29]:
# keep only columns that include venue name, and anything that is associated with location
Cafeteria_clean_columns = ['name', 'categories'] + [col for col in Cafeteria_dataframe.columns if col.startswith('location.')]+ ['id']
clean_Cafeteria_dataframe = Cafeteria_dataframe.loc[:,Cafeteria_clean_columns]

# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list4 = row['categories']
    except:
        categories_list4 = row['venue.categories']
        
    if len(categories_list4) == 0:
        return None
    else:
        return categories_list4[0]['name']

# filter the category for each row
clean_Cafeteria_dataframe['categories'] = clean_Cafeteria_dataframe.apply(get_category_type, axis=1)

# clean column names by keeping only last term
clean_Cafeteria_dataframe.columns = [column.split('.')[-1] for column in clean_Cafeteria_dataframe.columns]

clean_Cafeteria_dataframe.head()

,name,categories,address,cc,city,country,crossStreet,distance,formattedAddress,labeledLatLngs,lat,lng,postalCode,state,id
0,Cafeteria,Fast Food Restaurant,The Hospital for Sick Children (SickKids),CA,Toronto,Canada,NaN,448,"[The Hospital for Sick Children (SickKids), To...","[{'label': 'display', 'lat': 43.65720930549997...",43.657209,-79.386063,NaN,ON,4f9165bde4b04fef0087e4e2
1,Stay Cafeteria 慢走,Asian Restaurant,388 Spadina Ave,CA,Toronto,Canada,at Nassau St,1246,"[388 Spadina Ave (at Nassau St), Toronto ON M5...","[{'label': 'display', 'lat': 43.65545427446646...",43.655454,-79.399163,M5T 2G5,ON,528e99a211d262edc3708a6f
2,Sears Cafeteria,Food Court,Yonge,CA,Toronto,Canada,Dundas,387,"[Yonge (Dundas), Toronto ON, Canada]","[{'label': 'display', 'lat': 43.65603785909391...",43.656038,-79.380672,NaN,ON,4b57256bf964a520332828e3
3,Industrial Alliance Cafeteria,Cafeteria,NaN,CA,Toronto,Canada,NaN,505,"[Toronto ON, Canada]","[{'label': 'display', 'lat': 43.655657, 'lng':...",43.655657,-79.389443,NaN,ON,53bd6dfe498e6c1d3bc9333e
4,Chestnut Tree Cafeteria,Cafeteria,89 Chestnut Street,CA,Toronto,Canada,NaN,584,"[89 Chestnut Street, Toronto ON, Canada]","[{'label': 'display', 'lat': 43.648231, 'lng':...",43.648231,-79.384045,NaN,ON,524f1c8f498e983dde61ada0


In [30]:
# delete unnecessary columns
clean_Cafeteria_dataframe2= clean_Cafeteria_dataframe.drop(['cc', 'city', 'country', 'crossStreet', 'distance', 'formattedAddress',\
                                        'labeledLatLngs', 'id'], axis=1)
clean_Cafeteria_dataframe2.head()

,name,categories,address,lat,lng,postalCode,state
0,Cafeteria,Fast Food Restaurant,The Hospital for Sick Children (SickKids),43.657209,-79.386063,NaN,ON
1,Stay Cafeteria 慢走,Asian Restaurant,388 Spadina Ave,43.655454,-79.399163,M5T 2G5,ON
2,Sears Cafeteria,Food Court,Yonge,43.656038,-79.380672,NaN,ON
3,Industrial Alliance Cafeteria,Cafeteria,NaN,43.655657,-79.389443,NaN,ON
4,Chestnut Tree Cafeteria,Cafeteria,89 Chestnut Street,43.648231,-79.384045,NaN,ON


In [31]:
# delete rows with none values
df_Cafeteria = clean_Cafeteria_dataframe2.dropna(axis=0, how='any', thresh=None, subset=None, inplace=False)
df_Cafeteria.head()

,name,categories,address,lat,lng,postalCode,state
1,Stay Cafeteria 慢走,Asian Restaurant,388 Spadina Ave,43.655454,-79.399163,M5T 2G5,ON
9,De La Salle College Cafeteria,College Cafeteria,131 Farnham Avenue,43.683003,-79.397815,M4V 1H7,ON
18,Marketeria,Restaurant,30 Bond St.,43.653585,-79.378843,M5B 1W8,ON
20,The Hub,College Cafeteria,350 Victoria St.,43.658585,-79.380622,M5B 2K3,ON
27,6th Floor Cafeteria 12 Concorde Place,Café,12 Concorde Place,43.721861,-79.329339,m3c 3k7,ON


## Search for Shopping Stores

In [32]:
# search for Shopping
search_query = 'Shopping'
radius = 1000

# Define the corresponding URL
url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&query={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude, VERSION, search_query, radius, LIMIT)
url

'https://api.foursquare.com/v2/venues/search?client_id=4FQQJE2LMGVLERRRE2BOFR1LQQR5ZEHFLTD1WRAFKOFT4TI0&client_secret=RCSNR5TH31NQRG2QYCHK101HRMTUZHBB2TSJ1CJU5UOGRXRQ&ll=43.6534817,-79.3839347&v=20180604&query=Shopping&radius=1000&limit=30'

In [33]:
# Send the GET Request and examine the results
sresults = requests.get(url).json()
#sresults

In [34]:
# assign relevant part of JSON to venues
venues = sresults['response']['venues']

# tranform venues into a dataframe
Shopping_dataframe = json_normalize(venues)
Shopping_dataframe.head()

,categories,hasPerk,id,location.address,location.cc,location.city,location.country,location.crossStreet,location.distance,location.formattedAddress,location.labeledLatLngs,location.lat,location.lng,location.neighborhood,location.postalCode,location.state,name,referralId
0,"[{'id': '4bf58dd8d48988d1f6941735', 'name': 'D...",False,5840cafe06f1a34af80cc609,176 Yonge Street,CA,Toronto,Canada,Queen Street West,424,"[176 Yonge Street (Queen Street West), Toronto...","[{'label': 'display', 'lat': 43.65181, 'lng': ...",43.651810,-79.379192,NaN,M5C 2L7,ON,Saks Fifth Avenue Club - Personal Shopping,v-1589118153
1,"[{'id': '4bf58dd8d48988d1fd941735', 'name': 'S...",False,4b9e86dff964a520e6eb36e3,66 Wellington St W,CA,Toronto,Canada,btw York & Bay St,741,"[66 Wellington St W (btw York & Bay St), Toron...","[{'label': 'display', 'lat': 43.64718424611984...",43.647184,-79.380932,NaN,M5K 1A1,ON,TD Centre Shopping Concourse,v-1589118153
2,"[{'id': '4bf58dd8d48988d1fd941735', 'name': 'S...",False,4ad77a12f964a520260b21e3,220 Yonge St,CA,Toronto,Canada,btwn Queen & Dundas,287,"[220 Yonge St (btwn Queen & Dundas), Toronto O...",NaN,43.654540,-79.380677,"Downtown Toronto, Toronto, ON",M5B 2H1,ON,CF Toronto Eaton Centre,v-1589118153


## Clean Shopping Dataframe

In [35]:
# keep only columns that include venue name, and anything that is associated with location
Shopping_clean_columns = ['name', 'categories'] + [col for col in Shopping_dataframe.columns if col.startswith('location.')]+ ['id']
clean_Shopping_dataframe = Shopping_dataframe.loc[:,Shopping_clean_columns]

# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list5 = row['categories']
    except:
        categories_list5 = row['venue.categories']
        
    if len(categories_list5) == 0:
        return None
    else:
        return categories_list5[0]['name']

# filter the category for each row
clean_Shopping_dataframe['categories'] = clean_Shopping_dataframe.apply(get_category_type, axis=1)

# clean column names by keeping only last term
clean_Shopping_dataframe.columns = [column.split('.')[-1] for column in clean_Shopping_dataframe.columns]

clean_Shopping_dataframe.head()

,name,categories,address,cc,city,country,crossStreet,distance,formattedAddress,labeledLatLngs,lat,lng,neighborhood,postalCode,state,id
0,Saks Fifth Avenue Club - Personal Shopping,Department Store,176 Yonge Street,CA,Toronto,Canada,Queen Street West,424,"[176 Yonge Street (Queen Street West), Toronto...","[{'label': 'display', 'lat': 43.65181, 'lng': ...",43.651810,-79.379192,NaN,M5C 2L7,ON,5840cafe06f1a34af80cc609
1,TD Centre Shopping Concourse,Shopping Mall,66 Wellington St W,CA,Toronto,Canada,btw York & Bay St,741,"[66 Wellington St W (btw York & Bay St), Toron...","[{'label': 'display', 'lat': 43.64718424611984...",43.647184,-79.380932,NaN,M5K 1A1,ON,4b9e86dff964a520e6eb36e3
2,CF Toronto Eaton Centre,Shopping Mall,220 Yonge St,CA,Toronto,Canada,btwn Queen & Dundas,287,"[220 Yonge St (btwn Queen & Dundas), Toronto O...",NaN,43.654540,-79.380677,"Downtown Toronto, Toronto, ON",M5B 2H1,ON,4ad77a12f964a520260b21e3


In [36]:
# delete unnecessary columns
clean_Shopping_dataframe2= clean_Shopping_dataframe.drop(['cc', 'city', 'country', 'crossStreet', 'distance', 'formattedAddress',\
                                        'labeledLatLngs', 'neighborhood', 'id'], axis=1)
clean_Shopping_dataframe2.head()

,name,categories,address,lat,lng,postalCode,state
0,Saks Fifth Avenue Club - Personal Shopping,Department Store,176 Yonge Street,43.651810,-79.379192,M5C 2L7,ON
1,TD Centre Shopping Concourse,Shopping Mall,66 Wellington St W,43.647184,-79.380932,M5K 1A1,ON
2,CF Toronto Eaton Centre,Shopping Mall,220 Yonge St,43.654540,-79.380677,M5B 2H1,ON


In [37]:
# delete rows which its category is not Shopping Mall
df_Shopping = clean_Shopping_dataframe2[clean_Shopping_dataframe2.categories == 'Shopping Mall']
df_Shopping.head()

,name,categories,address,lat,lng,postalCode,state
1,TD Centre Shopping Concourse,Shopping Mall,66 Wellington St W,43.647184,-79.380932,M5K 1A1,ON
2,CF Toronto Eaton Centre,Shopping Mall,220 Yonge St,43.654540,-79.380677,M5B 2H1,ON


## Generate map to visualize hotels, shopping stores and Cafeteria and how they cluster together

In [38]:
# create dataframe of hotels, shopping stores and Cafeteria
hotel_neighbourhood_df = pd.concat([df_hotel, df_Cafeteria, df_Shopping], ignore_index=True)
hotel_neighbourhood_df.head()

,name,categories,address,lat,lng,postalCode,state
0,Sheraton Centre Toronto Hotel,Hotel,123 Queen Street West,43.650594,-79.384530,M5H 2M9,ON
1,Stay Cafeteria 慢走,Asian Restaurant,388 Spadina Ave,43.655454,-79.399163,M5T 2G5,ON
2,De La Salle College Cafeteria,College Cafeteria,131 Farnham Avenue,43.683003,-79.397815,M4V 1H7,ON
3,Marketeria,Restaurant,30 Bond St.,43.653585,-79.378843,M5B 1W8,ON
4,The Hub,College Cafeteria,350 Victoria St.,43.658585,-79.380622,M5B 2K3,ON


In [39]:
# Generate map to visualize hotel neighbourhood including shopping stores and Cafeteria 
hotel_map = folium.Map(location=[latitude, longitude], zoom_start=14)

for lat, lng, name, categories, address in zip(hotel_neighbourhood_df['lat'], hotel_neighbourhood_df['lng'], 
                                           hotel_neighbourhood_df['name'], hotel_neighbourhood_df['categories'],\
                                               hotel_neighbourhood_df['address']):
    label = '{}, {}'.format(name, address)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='blue',
        fill_opacity=0.7,
        parse_html=False).add_to(hotel_map)  
    
hotel_map

## Generate map to visualize Park, Restaurant and Cafeteria and how they cluster together

In [40]:
# create dataframe of Park, Restaurant and Cafeteria
park_neighbourhood_df = pd.concat([df_park, df_Restaurant, df_Cafeteria,], ignore_index=True)
park_neighbourhood_df.head()

,name,categories,address,lat,lng,postalCode,state
0,Queen's Park,Park,University Ave.,43.663946,-79.392180,M5R 2E8,ON
1,Riverdale Park West,Park,500 Gerrard St.,43.666048,-79.360941,M5A 2H3,ON
2,Trinity Bellwoods Park,Park,1053 Dundas St. W.,43.647072,-79.413756,M5H 2N2,ON
3,Riverdale Park East,Park,550 Broadview Ave,43.669951,-79.355493,M4K 2P1,ON
4,Coronation Park,Park,711 Lake Shore Blvd. W,43.634553,-79.405403,M5V 3T7,ON


In [41]:
# Generate map to visualize park neighbourhood including Restaurant and Cafeteria 
park_map = folium.Map(location=[latitude, longitude], zoom_start=14)

for lat, lng, name, categories, address in zip(park_neighbourhood_df['lat'], park_neighbourhood_df['lng'], 
                                           park_neighbourhood_df['name'], park_neighbourhood_df['categories'],\
                                               park_neighbourhood_df['address']):
    label = '{}, {}'.format(name, address)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='blue',
        fill_opacity=0.7,
        parse_html=False).add_to(park_map)  
    
park_map

### Notebook Link on Watson Studio: (in case maps are not visible)

https://dataplatform.cloud.ibm.com/analytics/notebooks/v2/61677a4d-c067-46c6-a317-ac3abce11ef8/view?access_token=f9c1c0187357b7fa0b50f52c3076d97f776d8afd5475e3fff8f500ecc1592e21